In [1]:
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta, date
import numpy as np
import os
import sys
import shutil
sys.path.append('../helper_functions')
import defillama_utils as dfl

In [2]:
# date ranges to build charts for
drange = [7, 14, 30, 90, 180, 365]
# Do we count net flows marked at the lastest token price (1) or the price on each day (0)
# By default, we opt to 1, so that price movement isn't accidentally counted as + or - flow remainder
mark_at_latest_price = 1 #some errors with missing token prices we need to find solves for first

trailing_num_days = max(drange)
# print(trailing_num_days)

start_date = datetime.utcnow().date()-timedelta(days=trailing_num_days +1)
print(start_date)

2022-09-12


In [3]:
#get all apps > 10 m tvl
min_tvl = 10_000_000

# if TVL by token is not available, do we fallback on raw TVL (sensitive to token prices)?
is_fallback_on_raw_tvl = True#False

df_df = dfl.get_all_protocol_tvls_by_chain_and_token(min_tvl, is_fallback_on_raw_tvl)

In [4]:

df_df.head()

# Test for errors
# df_df_all[(df_df_all['protocol'] == 'app_name') & (df_df_all['date'] == '2023-01-27')]

,date,token,token_value,usd_value,total_app_tvl,protocol,slug,chain,category,name,parent_protocol
0,2020-12-19 23:00:00,WETH,2272.26688,1.484681e+06,1.484681e+06,lido,lido,Ethereum,Liquid Staking,Lido,Lido
1,2020-12-19 23:00:00,MATIC,0,0.000000e+00,1.484681e+06,lido,lido,Ethereum,Liquid Staking,Lido,Lido
2,2020-12-20 23:00:00,WETH,4140.77143,2.697598e+06,2.697598e+06,lido,lido,Ethereum,Liquid Staking,Lido,Lido
3,2020-12-20 23:00:00,MATIC,0,0.000000e+00,2.697598e+06,lido,lido,Ethereum,Liquid Staking,Lido,Lido
4,2020-12-21 23:00:00,WETH,5562.49008,3.410254e+06,3.410254e+06,lido,lido,Ethereum,Liquid Staking,Lido,Lido


In [5]:
# display(df_df_all)

# df_df_all2['token_value'] = df_df_all2['token_value'].fillna(0)
df_df['token_value'] = df_df['token_value'].astype('float64')
df_df['usd_value'] = df_df['usd_value'].astype('float64')
# display(df_df_all2)

In [6]:
#create an extra day to handle for tokens dropping to 0

df_df = df_df.fillna(0)
df_df_shift = df_df.copy()
df_df_shift['date'] = df_df_shift['date'] + timedelta(days=1)
df_df_shift['token_value'] = 0.0
df_df_shift['usd_value'] = 0.0

#merge back in
df_df = pd.concat([df_df,df_df_shift])
df_df_shift = None #Free Up memory
# print(df_df_all.dtypes)

df_df = df_df[df_df['date'] <= pd.to_datetime("today") ]

df_df['token_value'] = df_df['token_value'].fillna(0)
df_df_all = df_df.groupby(['date','token','chain','protocol','name','category','parent_protocol']).sum(['usd_value','token_value'])


df_df = df_df.reset_index()

In [7]:
print("done api")

done api


In [8]:
#filter down a bit so we can do trailing comparisons w/o doing every row
df_df = df_df[df_df['date'].dt.date >= start_date-timedelta(days=1) ]

#trailing comparison
df_df['last_token_value'] = df_df.groupby(['token','protocol','chain'])['token_value'].shift(1)
#now actually filter
df_df = df_df[df_df['date'].dt.date >= start_date ]
# display(df_df[df_df['protocol'] == 'velodrome'])

In [9]:
data_df = df_df.copy()
df_df = None # Free up memory
data_df = data_df.sort_values(by='date')

# price = usd value / num tokens
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']
data_df['last_price_usd'] = data_df.groupby(['token','protocol', 'chain'])['price_usd'].shift(1)

# If first instnace of token, make sure there's no price diff
data_df['last_price_usd'] = data_df[['last_price_usd', 'price_usd']].bfill(axis=1).iloc[:, 0]
#Forward fill if token drops off
data_df['price_usd'] = data_df[['price_usd','last_price_usd']].bfill(axis=1).iloc[:, 0]

data_df.sample(10)

,index,date,token,token_value,usd_value,total_app_tvl,protocol,slug,chain,category,name,parent_protocol,last_token_value,price_usd,last_price_usd
2457788,1932180,2023-08-23,PNK,91931.13322,1776.95618,1.848494e+09,uniswap-v3,uniswap-v3,Ethereum,Dexes,Uniswap V3,parent#uniswap,91584.35519,0.019329,0.019329
13985566,5248,2022-10-23,DGB,0.00000,0.00000,1.766588e+06,renvm,renvm,Binance,Bridge,RenVM,RenVM,0.00000,0.008500,0.008500
6729440,519,2023-07-09,WETH,270.05174,503819.33238,1.002710e+06,paraspace-lending-v1,paraspace-lending-v1,Arbitrum,Lending,ParaSpace Lending V1,parent#parax,264.57732,1865.640015,1865.640015
5046161,41286,2023-07-07,RETH,0.10000,198.17700,5.468789e+07,bancor-v3,bancor-v3,Ethereum,Dexes,Bancor V3,parent#bancor,0.10000,1981.770000,1981.770000
13682351,3153,2023-06-29,WBTC,0.00000,0.00000,0.000000e+00,connext,connext,Moonbeam,Cross Chain,Connext,Connext,0.00000,NaN,NaN
5651224,2461,2022-12-15,FRAX,0.00000,0.00000,3.766725e+06,dforce,dforce,Ethereum-borrowed,Lending,dForce,dForce,0.00000,NaN,NaN
6822017,85868,2023-05-10,BAIL,32926.00000,1.36445,1.803831e+07,serum,serum,Solana,Dexes,Serum,Serum,32926.00000,0.000041,0.000041
11491449,3181,2023-02-26,UST,0.00000,0.00000,4.560000e-02,synapse,synapse,Moonbeam,Cross Chain,Synapse,Synapse,0.00000,NaN,NaN
11989790,63,2022-10-07,FRAX,0.00000,0.00000,1.975048e+06,frax-swap,frax-swap,Polygon,Dexes,Frax Swap,parent#frax-finance,0.00000,0.999893,0.999893
12307089,2166,2023-05-15,RDNT,0.00000,0.00000,2.464098e+07,joe-v2.1,joe-v2.1,Arbitrum,Dexes,Joe V2.1,parent#trader-joe,0.00000,NaN,NaN


In [10]:
# Find what is the latest token price. This sometimes gets skewed if tokens disappear or supply locked goes to 0
# Token's recency rank by chain - For calculating prices
data_df['token_rank_desc'] = data_df.groupby(['chain','token'])['date'].\
                            rank(method='dense',ascending=False).astype(int)
# Token's recency rank by chain & app - For calculating prices
data_df['token_rank_desc_prot'] = data_df.groupby(['chain','token','protocol'])['date'].\
                            rank(method='dense',ascending=False).astype(int)
# Token's recency rank by chain & app if > 0 - For calculating prices
# data_df['token_rank_desc_prot_gt0'] = data_df[data_df['token_value'] > 0]\
#                                     .groupby(['chain', 'token', 'protocol'])['date']\
#                                     .rank(method='first', ascending=False).astype(int)

# get latest price either by protocol or in aggregate
# if we don't have a match by protocol, then select in aggregate.
# This section is messy
prices_df = data_df[['chain','protocol','token']].drop_duplicates()


latest_prices_df_raw_prot = data_df.loc[~data_df['price_usd'].isna()][['token','chain','protocol','price_usd']][data_df['token_rank_desc_prot'] ==1]
latest_prices_df_prot = latest_prices_df_raw_prot.groupby(['token','chain','protocol']).median('price_usd')
latest_prices_df_prot = latest_prices_df_prot.rename(columns={'price_usd':'latest_price_usd_prot'})
latest_prices_df_prot = latest_prices_df_prot.reset_index()
prices_df = prices_df.merge(latest_prices_df_prot,on=['token','chain','protocol'], how='left')
latest_prices_df_raw_prot = None # Free up memory
print('done latest_prices_df_raw_prot')

latest_prices_df_raw = data_df.loc[~data_df['price_usd'].isna()][['token','chain','price_usd']][data_df['token_rank_desc'] ==1]
latest_prices_df = latest_prices_df_raw.groupby(['token','chain']).median('price_usd')
latest_prices_df = latest_prices_df.rename(columns={'price_usd':'latest_price_usd_raw'})
latest_prices_df = latest_prices_df.reset_index()
prices_df = prices_df.merge(latest_prices_df,on=['token','chain'], how='left')
latest_prices_df = None # Free up memory
print('done latest_prices_df')

prices_df['latest_price_usd_prot_gt0'] = 0
# latest_prices_df_raw_prot_gt0 = data_df.loc[~data_df['price_usd'].isna()][['token','chain','price_usd','protocol']][data_df['token_rank_desc_prot_gt0'] ==1]
# latest_prices_df_prot_gt0 = latest_prices_df_raw_prot_gt0.groupby(['token','chain','protocol']).median('price_usd')
# latest_prices_df_prot_gt0 = latest_prices_df_prot_gt0.rename(columns={'price_usd':'latest_price_usd_prot_gt0'})
# latest_prices_df_prot_gt0 = latest_prices_df_prot_gt0.reset_index()
# prices_df = prices_df.merge(latest_prices_df_prot_gt0,on=['token','chain','protocol'], how='left')
# latest_prices_df_prot_gt0 = None # Free up memory
# print('done latest_prices_df_prot_gt0')


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_83957/3261005683.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  latest_prices_df_raw_prot = data_df.loc[~data_df['price_usd'].isna()][['token','chain','protocol','price_usd']][data_df['token_rank_desc_prot'] ==1]


done latest_prices_df_raw_prot
done latest_prices_df


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_83957/3261005683.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  latest_prices_df_raw = data_df.loc[~data_df['price_usd'].isna()][['token','chain','price_usd']][data_df['token_rank_desc'] ==1]


In [11]:

# prices_df['latest_price_usd'] = \
#         prices_df['latest_price_usd_prot'].where(prices_df['latest_price_usd_prot'] > 0, \
#         prices_df['latest_price_usd_raw'].where(prices_df['latest_price_usd_raw'] > 0, \
#         prices_df['latest_price_usd_prot_gt0']))

#Select the latest price we want in priority order

# Chat GPT did this logic
# Define the conditions and choices for the numpy.select() function
conditions = [
    prices_df['latest_price_usd_prot'] > 0,
    prices_df['latest_price_usd_raw'] > 0,
]
choices = [
    prices_df['latest_price_usd_prot'],
    prices_df['latest_price_usd_raw'],
]

# If both conditions are False, choose prices_df['latest_price_usd_prot_gt0']
default_choice = prices_df['latest_price_usd_prot_gt0']

# Use numpy.select() to select the latest price
prices_df['latest_price_usd'] = np.select(conditions, choices, default_choice)
print("done price choice")

#Filter down
prices_df = prices_df[['chain','protocol','token','latest_price_usd']]

prices_df = prices_df[~prices_df['latest_price_usd'].isna()]

#Merge back in to the data dataframe
data_df = data_df.merge(prices_df,on=['token','chain','protocol'], how='left')
prices_df = None #Free Up memory
print("prices map done")

done price choice
prices map done


In [12]:
# Sort in date order

data_df.sort_values(by='date',inplace=True)

# get net token change
data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']
# get net token change * current price
data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']
# get net token change * latest price
data_df['net_dollar_flow_latest_price'] = data_df['net_token_flow'] * data_df['latest_price_usd']

#Filter out weird errors and things
data_df = data_df[abs(data_df['net_dollar_flow']) < 50_000_000_000] #50 bil error bar for bad prices
data_df = data_df[~data_df['net_dollar_flow'].isna()]

In [13]:
# Handle for errors where a token price went to zero (i.e. magpie ANKRBNB 2023-01-27)
data_df['net_dollar_flow_latest_price'] = np.where(
    data_df['net_dollar_flow'] == 0, 0, data_df['net_dollar_flow_latest_price']
)

In [14]:
# Get net flows by protocol

netdf_df = data_df[['date','protocol','chain','name','category','parent_protocol','net_dollar_flow','usd_value','net_dollar_flow_latest_price']]

data_df = None #Free Up memory

netdf_df = netdf_df.fillna(0)
netdf_df = netdf_df.sort_values(by='date',ascending=True)
netdf_df = netdf_df.groupby(['date','protocol','chain','name','category','parent_protocol']).sum(['net_dollar_flow','usd_value','net_dollar_flow_latest_price']) ##agg by app

#usd_value is the TVL on a given day
netdf_df = netdf_df.groupby(['date','protocol','chain','usd_value','name','category','parent_protocol']).sum(['net_dollar_flow','net_dollar_flow_latest_price'])

netdf_df.reset_index(inplace=True)
netdf_df.head()

# Drop index column if it exists
try:
        netdf_df.drop(columns=['index'],inplace=True)
except:
        pass
# display(netdf_df[netdf_df['protocol']=='makerdao'])



In [15]:
#get latest
netdf_df['rank_desc'] = netdf_df.groupby(['protocol', 'chain'])['date'].\
                            rank(method='dense',ascending=False).astype(int)
# display(netdf_df[netdf_df['protocol'] == 'lyra'])
netdf_df = netdf_df[  #( netdf_df['rank_desc'] == 1 ) &\
                        (~netdf_df['chain'].str.contains('-borrowed')) &\
                        (~netdf_df['chain'].str.contains('-staking')) &\
                        (~netdf_df['chain'].str.contains('-pool2')) &\
                            (~netdf_df['chain'].str.contains('-treasury')) &\
                        (~( netdf_df['chain'] == 'treasury') ) &\
                        (~( netdf_df['chain'] == 'borrowed') ) &\
                        (~( netdf_df['chain'] == 'staking') ) &\
                            (~( netdf_df['chain'] == 'treasury') ) &\
                        (~( netdf_df['chain'] == 'pool2') ) &\
                        (~( netdf_df['protocol'] == 'polygon-bridge-&-staking') )  &\
                            (~(netdf_df['protocol'].str[-4:] == '-cex') )
#                         & (~( netdf_df['chain'] == 'Ethereum') )
                        ]
# display(netdf_df[netdf_df['protocol']=='makerdao'])

In [16]:
# netdf_df.columns

for i in ('svg','png','html'):
                dir_path = "img_outputs/" + i
                # clear out folder
                if os.path.exists(dir_path):
                        shutil.rmtree(dir_path)
                # create folder
                os.mkdir(dir_path)

In [17]:
summary_df = netdf_df.copy()

summary_df = summary_df.sort_values(by='date',ascending=True)

# Mark at latest price if chosen
if mark_at_latest_price == 1:
        summary_df['mark_at_latest_price'] = mark_at_latest_price
        summary_df['net_dollar_flow'] = summary_df['net_dollar_flow_latest_price']
        titleval_append = ' - At Latest Prices'
else:
        titleval_append = ''

# Cast 'net_dollar_flow' to float64 data type
summary_df['net_dollar_flow'] = summary_df['net_dollar_flow'].astype('float64')

summary_df['cumul_net_dollar_flow'] = summary_df[['protocol','chain','net_dollar_flow']]\
                                    .groupby(['protocol','chain']).cumsum()
summary_df['flow_direction'] = np.where(summary_df['cumul_net_dollar_flow']*1.0 >= 0, 1,-1)
summary_df['abs_cumul_net_dollar_flow'] = abs(summary_df['cumul_net_dollar_flow'])

for i in drange:
        if i == 0:
                continue
                # summary_df['cumul_net_dollar_flow'] = summary_df[['protocol','chain','net_dollar_flow']]\
                #                     .groupby(['protocol','chain']).cumsum()
                # summary_df['flow_direction'] = np.where(summary_df['cumul_net_dollar_flow']*1.0 >= 0, 1,-1)
                # summary_df['abs_cumul_net_dollar_flow'] = abs(summary_df['cumul_net_dollar_flow'])

        else:
                col_str = 'cumul_net_dollar_flow_' + str(i) + 'd'
                tvl_str = 'daily_avg_tvl_' + str(i) + 'd'
                
                #chatgpt version
                summary_df[col_str] = summary_df.groupby(['protocol','chain'])['net_dollar_flow']\
                                        .apply(lambda x: x.rolling(i, min_periods=1).sum())

                summary_df[tvl_str] = summary_df[['protocol','chain','usd_value']]\
                                    .groupby(['protocol','chain'])['usd_value'].transform(lambda x: x.rolling(i, min_periods=1).mean() )
                
                summary_df['flow_direction_' + str(i) + 'd'] = np.where(summary_df[col_str]*1.0 >= 0, 1, -1)
                summary_df['abs_cumul_net_dollar_flow_' + str(i) + 'd'] = abs(summary_df[col_str])

summary_df['pct_of_tvl'] = 100* summary_df['net_dollar_flow'] / summary_df['usd_value']
final_summary_df = summary_df[(summary_df['rank_desc'] == 1) & (summary_df['date'] >= pd.to_datetime("today") -timedelta(days=7))]
final_summary_df = final_summary_df[final_summary_df['cumul_net_dollar_flow']< 1e20] #weird error handling


os.makedirs('csv_outputs', exist_ok=True)
os.makedirs('img_outputs', exist_ok=True)
os.makedirs('img_outputs/png', exist_ok=True)
os.makedirs('img_outputs/svg', exist_ok=True)
os.makedirs('img_outputs/html', exist_ok=True)

final_summary_df.to_csv('csv_outputs/latest_tvl_app_trends.csv', mode='w', index=False, encoding='utf-8')


Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Aiohttp retry client was not closed
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7ff3299defe0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7ff329996980>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7ff348e0ff40>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7ff348e0edd0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7ff318e76110>
Unclosed client session
client_session: <aiohttp.cli

In [18]:
print("starting chart outputs")
# display(summary_df)
for i in drange:
        fig = ''
        if i == 0:
                yval = 'abs_cumul_net_dollar_flow'
                hval = 'cumul_net_dollar_flow'
                cval = 'flow_direction'
                saveval = 'net_app_flows'
                saveval_app = 'net_app_flows_by_app'
                titleval = "App Net Flows Change by Chain -> App - Last " + str(trailing_num_days) + \
                            " Days - (Apps with > $" + str(min_tvl/1e6) + "M TVL Shown)" + titleval_append
                titleval_app = "App Net Flows Change by App -> Chain - Last " + str(trailing_num_days) + \
                            " Days - (Apps with > $" + str(min_tvl/1e6) + "M TVL Shown)" + titleval_append
        else:
                yval = 'abs_cumul_net_dollar_flow_' + str(i) +'d'
                hval = 'cumul_net_dollar_flow_' + str(i) +'d'
                cval = 'flow_direction_' + str(i) +'d'
                saveval = 'net_app_flows_' + str(i) +'d'
                saveval_app = 'net_app_flows_by_app_' + str(i) +'d'
                titleval = "App Net Flows Change by Chain -> App - Last " + str(i) + \
                            " Days - (Apps with > $" + str(min_tvl/1e6) + "M TVL Shown)" + titleval_append
                titleval_app = "App Net Flows Change by App -> Chain - Last " + str(i) + \
                                " Days - (Apps with > $" + str(min_tvl/1e6) + "M TVL Shown)" + titleval_append
        if is_fallback_on_raw_tvl:
                subtitle = "<br><sup>Source: DefiLlama API | *For apps where DefiLlama didn't have flows by token, we use their total change in TVL (including token price change)</sup>"
        else:
                subtitle = ""

        fig = px.treemap(final_summary_df[final_summary_df[yval] !=0], \
                 path=[px.Constant("all"), 'chain', 'protocol'], \
#                  path=[px.Constant("all"), 'token', 'chain', 'protocol'], \
                 values=yval, color=cval
#                 ,color_discrete_map={'-1':'red', '1':'green'})
                ,color_continuous_scale='Spectral'
                     , title = titleval + subtitle
                
                , hover_data = [hval]
                )
        
        fig_app = px.treemap(final_summary_df[final_summary_df[yval] !=0], \
                 path=[px.Constant("all"), 'protocol','chain'], \
#                  path=[px.Constant("all"), 'token', 'chain', 'protocol'], \
                 values=yval, color=cval
#                 ,color_discrete_map={'-1':'red', '1':'green'})
                ,color_continuous_scale='Spectral'
                     , title = titleval_app + subtitle
                
                , hover_data = [hval]
                )

        fig.update_traces(root_color="lightgrey")
        fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
        fig_app.update_traces(root_color="lightgrey")
        fig_app.update_layout(margin = dict(t=50, l=25, r=25, b=25))

        fig.write_image("img_outputs/svg/" + saveval + ".svg") #
        fig.write_image("img_outputs/png/" + saveval + ".png") #
        fig.write_html("img_outputs/html/" + saveval + ".html", include_plotlyjs='cdn')

        fig_app.write_image("img_outputs/svg/" + saveval_app + ".svg") #
        fig_app.write_image("img_outputs/png/" + saveval_app + ".png") #
        fig_app.write_html("img_outputs/html/" + saveval_app + ".html", include_plotlyjs='cdn')

        if i == 30:
                fig.show()
# fig.data[0].textinfo = 'label+text+value'
# fig.update_layout(tickprefix = '$')

starting chart outputs


In [19]:
# ! jupyter nbconvert --to python total_app_net_flows_async.ipynb